In [1]:
import folium
from folium.features import DivIcon
from folium.plugins import MarkerCluster, MousePosition
import io
import pandas as pd
from math import sin, cos, sqrt, atan2, radians
import requests  # <-- added

# --- Load CSV from URL ---
URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'

# Use requests instead of js.fetch
resp = requests.get(URL)
resp.raise_for_status()  # optional: will raise error if request failed
spacex_csv_file = io.BytesIO(resp.content)
spacex_df = pd.read_csv(spacex_csv_file)

# --- Clean/normalize column names ---
spacex_df.columns = spacex_df.columns.str.strip().str.replace(' ', '_')

# --- Keep only needed columns ---
spacex_df = spacex_df[['Launch_Site', 'Lat', 'Long', 'class']]

# --- Add marker color based on success/failure ---
spacex_df['marker_color'] = spacex_df['class'].apply(lambda x: 'green' if x == 1 else 'red')

# --- Group launch sites for markers ---
launch_sites_df = spacex_df.groupby('Launch_Site').first().reset_index()

# --- NASA Johnson Space Center coordinates ---
nasa_coordinate = [29.559684888503615, -95.0830971930759]

# --- Initialize map ---
site_map = folium.Map(location=nasa_coordinate, zoom_start=5, tiles='OpenStreetMap')

# --- Add launch site markers (blue circles + labels) ---
for _, row in launch_sites_df.iterrows():
    coord = [row['Lat'], row['Long']]
    site_name = row['Launch_Site']

    # Blue circle
    folium.Circle(
        location=coord,
        radius=1000,
        color='blue',
        fill=True
    ).add_child(folium.Popup(site_name)).add_to(site_map)

    # Label
    folium.Marker(
        location=coord,
        icon=DivIcon(
            icon_size=(20,20),
            icon_anchor=(0,0),
            html=f'<div style="font-size:12px;color:blue;"><b>{site_name}</b></div>'
        )
    ).add_to(site_map)

# --- Add NASA JSC marker ---
folium.Circle(
    nasa_coordinate,
    radius=1000,
    color='#d35400',
    fill=True
).add_child(folium.Popup('NASA Johnson Space Center')).add_to(site_map)

folium.Marker(
    location=nasa_coordinate,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size:12px;color:#d35400;"><b>NASA JSC</b></div>'
    )
).add_to(site_map)

# --- Add MarkerCluster for all launches ---
marker_cluster = MarkerCluster().add_to(site_map)

for _, row in spacex_df.iterrows():
    coord = [row['Lat'], row['Long']]
    popup_text = f"{row['Launch_Site']} - {'Success' if row['class']==1 else 'Failure'}"

    folium.Marker(
        location=coord,
        popup=popup_text,
        icon=folium.Icon(color='white', icon_color=row['marker_color'], icon='info-sign')
    ).add_to(marker_cluster)

# --- Add MousePosition for coordinate exploration ---
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: -120.62516 ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat: 34.6352',
    lat_formatter=formatter,
    lng_formatter=formatter,
)
site_map.add_child(mouse_position)

# --- Distance calculation function ---
def calculate_distance(lat1, lon1, lat2, lon2):
    R = 6373.0  # km
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1-a))
    return R * c

# --- Example: Add a coastline point and visualize distance ---
launch_site_coord = [28.5623, -80.5770]      # Example launch site
coastline_coord = [28.56367, -80.57163]      # Closest coastline (found with MousePosition)
distance_km = calculate_distance(launch_site_coord[0], launch_site_coord[1],
                                 coastline_coord[0], coastline_coord[1])

# Marker at coastline showing distance
distance_marker = folium.Marker(
    location=coastline_coord,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size:12px;color:#d35400;"><b>{:.2f} KM</b></div>'.format(distance_km)
    )
)
site_map.add_child(distance_marker)

# Line connecting launch site and coastline
line = folium.PolyLine(locations=[launch_site_coord, coastline_coord], weight=2, color='blue')
site_map.add_child(line)

# --- Display the map ---
site_map


In [2]:
import folium
from folium.features import DivIcon
from math import radians, sin, cos, sqrt, atan2

# Function to calculate distance
def distance_km(lat1, lon1, lat2, lon2):
    R = 6373.0
    lat1, lon1, lat2, lon2 = map(radians, (lat1, lon1, lat2, lon2))
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = sin(dlat/2)**2 + cos(lat1)*cos(lat2)*sin(dlon/2)**2
    return R * 2 * atan2(sqrt(a), sqrt(1 - a))

# Launch site
launch_site = [28.5620, -80.5772]  # CCAFS SLC-40
city_coord = [28.6122, -80.8076]   # Titusville

# Distance
dist_city = distance_km(launch_site[0], launch_site[1], city_coord[0], city_coord[1])

# Map
map_city = folium.Map(location=launch_site, zoom_start=10, tiles='OpenStreetMap')

# Launch site marker
folium.Marker(
    location=launch_site,
    popup='CCAFS SLC-40',
    icon=DivIcon(icon_size=(20,20), icon_anchor=(0,0),
                 html='<div style="font-size:12px;color:#0000ff;"><b>CCAFS SLC-40</b></div>')
).add_to(map_city)

# City marker with distance
folium.Marker(
    location=city_coord,
    popup='Titusville',
    icon=DivIcon(icon_size=(20,20), icon_anchor=(0,0),
                 html=f'<div style="font-size:12px;color:#d35400;"><b>{dist_city:.2f} km</b></div>')
).add_to(map_city)

# Line
folium.PolyLine([launch_site, city_coord], color='green', weight=2).add_to(map_city)

# Save map to HTML
map_city.save("CCAFS_proximity_city.html")


In [6]:
# Coastline coordinate
coast_coord = [28.565, -80.56835]  # Atlantic Coast

dist_coast = distance_km(launch_site[0], launch_site[1], coast_coord[0], coast_coord[1])

map_coast = folium.Map(location=launch_site, zoom_start=10, tiles='OpenStreetMap')

folium.Marker(
    location=launch_site,
    popup='CCAFS SLC-40',
    icon=DivIcon(icon_size=(20,20), icon_anchor=(0,0),
                 html='<div style="font-size:12px;color:#0000ff;"><b>CCAFS SLC-40</b></div>')
).add_to(map_coast)

folium.Marker(
    location=coast_coord,
    popup='Atlantic Coast',
    icon=DivIcon(icon_size=(20,20), icon_anchor=(0,0),
                 html=f'<div style="font-size:12px;color:#d35400;"><b>{dist_coast:.2f} km</b></div>')
).add_to(map_coast)

folium.PolyLine([launch_site, coast_coord], color='blue', weight=2).add_to(map_coast)

map_coast.save("CCAFS_proximity_coastline.html")


In [7]:
# Railway coordinate
rail_coord = [28.5650, -80.5865]  # Example nearby railway

dist_rail = distance_km(launch_site[0], launch_site[1], rail_coord[0], rail_coord[1])

map_rail = folium.Map(location=launch_site, zoom_start=10, tiles='OpenStreetMap')

folium.Marker(
    location=launch_site,
    popup='CCAFS SLC-40',
    icon=DivIcon(icon_size=(20,20), icon_anchor=(0,0),
                 html='<div style="font-size:12px;color:#0000ff;"><b>CCAFS SLC-40</b></div>')
).add_to(map_rail)

folium.Marker(
    location=rail_coord,
    popup='Nearby Railway',
    icon=DivIcon(icon_size=(20,20), icon_anchor=(0,0),
                 html=f'<div style="font-size:12px;color:#d35400;"><b>{dist_rail:.2f} km</b></div>')
).add_to(map_rail)

folium.PolyLine([launch_site, rail_coord], color='red', weight=2).add_to(map_rail)

map_rail.save("CCAFS_proximity_rail.html")
